In [ ]:
import io
import zipfile



# Unzip the file.
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Copy of door_open_door_close_door_stop_max_1sec.zip")
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.utils import to_categorical

In [ ]:
DATASET_PATH = '/tmp/hackathon'

data_dir = pathlib.Path(DATASET_PATH)
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[(commands != 'README.md') & (commands != '.DS_Store')]
print('Commands Available:', commands)


Commands Available: ['door_open' 'door_close' 'door_stop']


In [ ]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both')

label_names = np.array(train_ds.class_names)
print()
print("label :", label_names)


Found 12674 files belonging to 3 classes.
Using 10140 files for training.
Using 2534 files for validation.

label : ['door_close' 'door_open' 'door_stop']


In [ ]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)


In [ ]:
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)


In [ ]:
def get_spectrogram(waveform):
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension so that the Spectrogram can be used
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram


In [ ]:
def make_spec_ds(ds):
  return ds.map(
      map_func=lambda audio,label: (get_spectrogram(audio), label),
      num_parallel_calls=tf.data.AUTOTUNE)

train_spectrogram_ds = make_spec_ds(train_ds)
val_spectrogram_ds = make_spec_ds(val_ds)
test_spectrogram_ds = make_spec_ds(test_ds)


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=(124, 129, 1)))
model.add(layers.Conv2D(16, (3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 122, 127, 16)      160       
                                                                 
 conv2d_9 (Conv2D)           (None, 120, 125, 16)      2320      
                                                                 
 flatten_4 (Flatten)         (None, 240000)            0         
                                                                 
 dense_8 (Dense)             (None, 128)               30720128  
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 30722737 (117.20 MB)
Trainable params: 30722737 (117.20 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compiling and fitting the model
model.compile('Adam', loss='BinaryCrossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

model.fit(train_spectrogram_ds, epochs=10, validation_data=test_spectrogram_ds)


Epoch 1/10
159/159 [==============================] - 330s 2s/step - loss: -46186.6172 - recall_4: 1.0000 - precision_4: 0.7249 - val_loss: -311943.5000 - val_recall_4: 1.0000 - val_precision_4: 0.7242
Epoch 2/10
159/159 [==============================] - 333s 2s/step - loss: -3016546.7500 - recall_4: 1.0000 - precision_4: 0.7249 - val_loss: -9233793.0000 - val_recall_4: 1.0000 - val_precision_4: 0.7242
Epoch 3/10
159/159 [==============================] - 323s 2s/step - loss: -27248052.0000 - recall_4: 1.0000 - precision_4: 0.7249 - val_loss: -59191968.0000 - val_recall_4: 1.0000 - val_precision_4: 0.7242
Epoch 4/10
159/159 [==============================] - 329s 2s/step - loss: -115191208.0000 - recall_4: 1.0000 - precision_4: 0.7249 - val_loss: -201839664.0000 - val_recall_4: 1.0000 - val_precision_4: 0.7242
Epoch 5/10
159/159 [==============================] - 317s 2s/step - loss: -319505792.0000 - recall_4: 1.0000 - precision_4: 0.7249 - val_loss: -498809184.0000 - val_recall_4: 1

In [ ]:
# Prediction for a data

audio, labels = train_ds.take(2)

yhat = model.predict(audio)


In [ ]:
for i in zip(audio,y_hat):
    print(i[0].shape)
    plt.figure(figsize=(30,20))
    plt.text(0, 0, "Predicted Class = "+str(i[1])[0],fontsize = 33,
         bbox = dict(facecolor = 'red', alpha = 0.2))
    plt.imshow(np.transpose(i[0])[0])
    plt.show()


In [ ]:
model.save('my_model_CNN-1937.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Using librose ms

In [ ]:
import librosa
import numpy as np


In [ ]:
import os

directory_path = "/tmp/hackathon/"  # Replace with the actual path

# Option 1: Using os.listdir()
folders = os.listdir(directory_path)


In [ ]:
pad2d = lambda a, i: a[:, 0: i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0],i - a.shape[1]))))

In [ ]:
def process_audio_file(label,filename):
    # Load audio
    audio, sr = librosa.load(directory_path+label+"/"+filename)
    print(sr)
    print(audio.shape)
    # Preprocess audio (optional)
    audio = librosa.util.normalize(audio)  # Normalize volume

    # Extract features
    spectrogram = librosa.stft(audio)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr)  # Example feature extraction

    return  mfccs, filename  # Return multiple features and filename

In [ ]:
audio_data = []
labels = []
for labelname in folders:
  files=os.listdir(directory_path+labelname+"/")
  for filename in files:
      mfccs, label = process_audio_file(labelname,filename)
      padded_mfcc = pad2d(mfccs,40)
      audio_data.append(padded_mfcc)
      z=0
      if labelname=='door_close':
        z=1
      if labelname=='door_stop':
        z=2
      labels.append(z)  # Assuming labels are extracted from filenames



In [ ]:
audio_data=np.array(audio_data)
labels=to_categorical(np.array(labels))

In [ ]:
train_X = np.expand_dims(audio_data, -1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(audio_data, labels, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model


In [ ]:

inputs = Input(shape=(X_train[0].shape))

# Recurrent layers for sequence processing
x = LSTM(64, return_sequences=True)(inputs)  # First LSTM layer
x = LSTM(32)(x)  # Second LSTM layer
x = LSTM(64, return_sequences=True, dropout=0.2)(inputs)  # Dropout rate of 20%
x = LSTM(32, dropout=0.2)(x)
# Output layer for categorical classification
outputs = Dense(3, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
254/254 [==============================] - 7s 26ms/step - loss: 0.1883 - accuracy: 0.9308 - val_loss: 0.1992 - val_accuracy: 0.9290
Epoch 2/30
254/254 [==============================] - 9s 35ms/step - loss: 0.1944 - accuracy: 0.9295 - val_loss: 0.1948 - val_accuracy: 0.9246
Epoch 3/30
254/254 [==============================] - 6s 26ms/step - loss: 0.1978 - accuracy: 0.9282 - val_loss: 0.2051 - val_accuracy: 0.9246
Epoch 4/30
254/254 [==============================] - 9s 36ms/step - loss: 0.1853 - accuracy: 0.9351 - val_loss: 0.1888 - val_accuracy: 0.9290
Epoch 5/30
254/254 [==============================] - 6s 25ms/step - loss: 0.1662 - accuracy: 0.9406 - val_loss: 0.1728 - val_accuracy: 0.9423
Epoch 6/30
254/254 [==============================] - 8s 32ms/step - loss: 0.1670 - accuracy: 0.9380 - val_loss: 0.2202 - val_accuracy: 0.9231
Epoch 7/30
254/254 [==============================] - 7s 27ms/step - loss: 0.1709 - accuracy: 0.9368 - val_loss: 0.2019 - val_accuracy: 0.9329

In [ ]:
model=tf.keras.models.load_model('/content/my-lstm.h5')
data, lab = process_audio_file('door_open','62d61e14449ba87643e73792.wav')
data = pad2d(data,40)
out=model.predict(tf.expand_dims(data, axis=0) )
percentage_list = []
for i in range(len(out[0])):
    percentage_list.append("{0:.2%}".format(out[0][i]))
classes=['door_open','door_close','door_stop']
my_result = list(zip(classes, percentage_list))
for i in range(len(my_result)):
    print(my_result[i])

22050
(22050,)
1/1 [==============================] - 2s 2s/step
('door_open', '100.00%')
('door_close', '0.00%')
('door_stop', '0.00%')


In [ ]:
model.save("my-lstm.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow import lite
model=tf.keras.models.load_model('/content/my-lstm.h5')
converter = lite.TFLiteConverter.from_keras_model( model ) # Your model's name
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
model = converter.convert()
file = open( 'my-lstm.tflite' , 'wb' )
file.write( model )

176624

In [ ]:
interpreter = tf.lite.Interpreter(model_path="/content/my-lstm.tflite")

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [ ]:
interpreter.allocate_tensors()

print(input_details)
# output details
print(output_details)

[{'name': 'serving_default_input_7:0', 'index': 0, 'shape': array([ 1, 20, 40], dtype=int32), 'shape_signature': array([-1, 20, 40], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 58, 'shape': array([1, 3], dtype=int32), 'shape_signature': array([-1,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
data, lab = process_audio_file('door_close','62d71c08449ba86a10f47158.wav')
data = pad2d(data,40)
interpreter.set_tensor(input_details[0]['index'], [data])

In [ ]:
interpreter.invoke()

In [ ]:
 # output_details[0]['index'] = the index which provides the input
output_data = interpreter.get_tensor(output_details[0]['index'])

output_data

array([[3.8604941e-03, 9.9537969e-01, 7.5987558e-04]], dtype=float32)